<a href="https://colab.research.google.com/github/tropical72/TensorFlowDeveloperCerti/blob/master/NLP_Week4_Exercise_Shakespeare_Answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>